<h1 style="text-align: center;"><strong>PROYECTO FINAL VISI&Oacute;N DE M&Aacute;QUINA</strong></h1>
<h2 style="text-align: center;"><br />M&Oacute;DULO ENTRENAMIENTO</h2>
<h4 style="text-align: center;"><br />Diego Fernando Godoy Rojas - Luis Carlos Fajardo Diaz</h4>
<h4 style="text-align: center;">Universidad Nacional De Colombia</h4>


# 7. Definimos la ubicación de los modelos

Definimos la ruta donde se guardara el modelo empleado y sus entrenamientos, en esta ocasión emplearemos un modelo pre-entrenado de TensorFlow para el entrenamiento de nuestro modelo de detección de objetos:

In [1]:
import os
#
CUSTOM_MODEL_NAME = 'my_ssd_mobnet' 
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [2]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow'),#'models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'), 
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [3]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [4]:
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

# 8. Preparación de TensorFlow

A continuación prepararemos el entorno de ejecución del proyecto:
- Descargamos el modelo pre-entrenado de los repositorios de Tensorflow.
- Instalamos todos los modulos y librerias requeridas para poder ejecutar Tensorflow.
- Ejecutamos el modulo de detección de objetos de Tensorflow TFOD.

In [5]:
#Instalamos wget para poder descargar archivos de internet directamente desde el notebook
if os.name=='nt':
    !pip install wget
    import wget

In [6]:
# Modulo para clonar paquetes de git-hub
!pip install git-clone

In [7]:
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
  !git-clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}

34840.0 KB, 18400 KB/s, 1.89 seconds passed         
https://github.com/tensorflow/models downloaded


In [8]:
# Instalamos TFOD
if os.name=='posix':  
    !apt-get install protobuf-compiler
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
    
if os.name=='nt':
    url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
    wget.download(url)
    !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
    !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
    os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
    !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
    !cd Tensorflow/models/research/slim && pip install -e . 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 40 not upgraded.
Processing /content/Tensorflow/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 9.7 MB 15.1 MB/s 
     |████████████████████████████████| 352 kB 45.4 MB/s 
     |████████████████████████████████| 1.6 MB 41.5 MB/s 
     |██████████████

In [9]:
VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'models','research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# Realizamos una verificación para ver que todo se encuentre debidamente instalado
!python {VERIFICATION_SCRIPT}

2021-08-18 05:26:08.961239: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-18 05:26:09.573350: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-18 05:26:09.574294: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.7.11: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2021-08-18 05:26:09.593209: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021

In [ ]:
# Usar solo si en la verificación se solicita
!pip install tensorflow --upgrade
!pip install Pillow
!python -m pip install pyyaml
!pip uninstall protobuf matplotlib -y
!pip install protobuf matplotlib==3.2

In [10]:
import object_detection

In [11]:
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2021-08-18 05:26:47--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.140.128, 2a00:1450:400c:c08::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.140.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  30.6MB/s    in 0.6s    

2021-08-18 05:26:49 (30.6 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-

# 9. Creamos el mapa de etiquetas

A continuación definimos las etiquetas que se asignaran mediante un diccionario a los objetos detectados:

In [12]:
labels = [{'name':'TarjetaCredito', 'id':1}, {'name':'Cedula', 'id':2}, {'name':'5mil', 'id':3}, {'name':'2mil', 'id':4}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 10. Convertimos los datos a archivos de TensorFlow

A partir de los conjuntos de imagenes de train y test creamos los registros en formato admitido para Tensorflow y de esta manera poder ser usados en el entrenamiento del modelo:

(Si se trabaja en colab se debe subir el archivo comprimido de imagenes a la carpeta images dentro de la carpeta de Workspace)

In [13]:
# SOLO CORRERLO SI SE TRABAJA EN COLAB!!
ARCHIVE_FILES = os.path.join(paths['IMAGE_PATH'], 'archive.tar.gz')
if os.path.exists(ARCHIVE_FILES):
    !tar -zxvf {ARCHIVE_FILES}

Tensorflow/workspace/images/train/
Tensorflow/workspace/images/train/2mil.011ca00c-ff0a-11eb-85d6-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.011ca00c-ff0a-11eb-85d6-c0e434d7a91c.xml
Tensorflow/workspace/images/train/2mil.038757dc-ff0a-11eb-8738-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.038757dc-ff0a-11eb-8738-c0e434d7a91c.xml
Tensorflow/workspace/images/train/2mil.05f290bd-ff0a-11eb-ac62-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.05f290bd-ff0a-11eb-ac62-c0e434d7a91c.xml
Tensorflow/workspace/images/train/2mil.0728446d-ff0a-11eb-815c-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.0728446d-ff0a-11eb-815c-c0e434d7a91c.xml
Tensorflow/workspace/images/train/2mil.085e172b-ff0a-11eb-96d7-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.085e172b-ff0a-11eb-96d7-c0e434d7a91c.xml
Tensorflow/workspace/images/train/2mil.099445ae-ff0a-11eb-8efa-c0e434d7a91c.jpg
Tensorflow/workspace/images/train/2mil.099445ae-ff0a-11eb-8efa-c0e434d7a91c.xml
Tenso

Si se trabaja en Colab se debe subir el archivo **generate_tfrecord.py** dentro de la carpeta scripts:

In [ ]:
#if not os.path.exists(files['TF_RECORD_SCRIPT']):
  #!git-clone https://github.com/nicknochnack/GenerateTFRecord {paths['SCRIPTS_PATH']}

Creamos los registros de train y test:

In [14]:
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/test.record


In [16]:
!pip install pytz

# 11. Copiamos la configuración del modelo en la carpeta de entrenamiento

In [17]:
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 12. Configuramos para realizar Transfer Learning

Se utilizara el modelo pre-entrenado para realizar el entrenamiento por transfer learning a nuestro modelo:

In [ ]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [19]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [20]:
#config

In [21]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [22]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [23]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 13. Entrenamiento del modelo!

Entrenamos nuestro modelo:

In [24]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'],'models' ,'research', 'object_detection', 'model_main_tf2.py')

In [25]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=10000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])
#print(command)

In [26]:
!{command}

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.6.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
2021-08-18 05:29:30.795508: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-18 05:29:30.812656: I tensorflow/stream_executor/cuda/cu

Una vez entrenado el modelo que utilizaremos, procedemos a comprimirlo para ser usado nuevamente en el entorno local

In [ ]:
!tar -czf models.tar.gz {paths['CHECKPOINT_PATH']}